**Social Media Addiction**

In [ ]:
# import manipulation libraries
import pandas as pd
import numpy as np

# import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# import machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# -------------------- DATA INGESTION --------------------
def data_ingestion():
    df = pd.read_csv(r'C:\SocialMediaAddiction_Analysis\data\raw\Social_Media_Addiction.csv')
    return df

In [ ]:
df = data_ingestion()
df.head()

In [ ]:
df = df.drop(columns=['Student_ID','Relationship_Status'], errors="ignore")


In [ ]:
# -------------------- ADDICTION LEVEL --------------------
def addiction_lavel(score):
    if score <= 3:
        return "Low"
    elif score <= 6:
        return "Medium"
    else:
        return "High"

In [ ]:
# -------------------- DATA EXPLORATION --------------------
def data_exploration(df):

    numerical_col = df.select_dtypes(exclude='object').columns
    categorical_col = df.select_dtypes(include = 'object').columns

    numerical_stats = []
    categorical_stats = []

   # ---------- Numerical Analysis ----------
    Q1 = df[numerical_col].quantile(0.25)
    Q3 = df[numerical_col].quantile(0.75)
    IQR = Q3 - Q1
    LW = Q1 - 1.5 * IQR
    UW = Q3 + 1.5 * IQR
    outlier_count = ((df[numerical_col] < LW) | (df[numerical_col] > UW))


    for col in numerical_col:
        num_stats = ({
            "feature": col,
            "mean": df[col].mean(),
            "median": df[col].median(),
            "quartile_1": Q1[col],
            "quartile_3": Q3[col],
            "IQR": IQR[col],
            "lower_whisker": LW[col],
            "upper_whisker": UW[col],
            "outlier_count": outlier_count[col].sum(),
            "std_dev": df[col].std(),
            "variance": df[col].var(),
            "skewness": df[col].skew(),
            "kurtosis": df[col].kurtosis()
        })

        numerical_stats.append(num_stats)

    # ---------- Categorical Analysis ----------

    for col in categorical_col:
        cat_stats = ({
            "feature": col,
            "unique_values": df[col].nunique(),
            "mode": df[col].mode()[0] if not df[col].mode().empty else None,
            "missing_values": df[col].isnull().sum()
        })

        categorical_stats.append(cat_stats)

    # return both reports
    numerical_report = pd.DataFrame(numerical_stats)
    categorical_report = pd.DataFrame(categorical_stats)

    return numerical_report, categorical_col    
        

    

In [ ]:
# -------------------- PREPROCESSOR --------------------
def data_preprocessor(df):

    categorical_col = df.select_dtypes(include='object').columns
    numerical_col = df.select_dtypes(exclude='object').columns

    numerical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num", numerical_pipeline, numerical_col),
        ("cat", categorical_pipeline, categorical_col)
    ])

    return preprocessor

In [ ]:
# -------------------- MODEL BUILD --------------------
def model_build(df):

    X = df.drop(columns=["Addicted", "addiction_lavel"], errors='ignore')
    y = df["Addicted"]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.2
    )

   # Preprocessing (handle categorical + scaling)
    preprocessor = data_preprocessor(X_train)

    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(),
        "XGBoost": XGBClassifier(use_label_encoder=False)
    }

    return models, X_train, X_test, y_train, y_test

In [ ]:
# -------------------- MODEL EVALUATION --------------------
def model_evalution(models, X_train, X_test, y_train, y_test):

    for model_name, model in models.items():

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        print(f"\n===== {model_name} ===")
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Classification Report:\n", classification_report(y_test, y_pred))
        print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
        print("-" * 50)

        

In [21]:
# -------------------- MAIN FUNCTION --------------------
def main():

    # 1 Data Ingestion
    df = data_ingestion()

    # Create 'addiction_lavel' and 'Addicted' columns (moved from global scope)
    df["addiction_lavel"] = df["Addicted_Score"].apply(addiction_lavel)
    df["Addicted"] = df["addiction_lavel"].apply(lambda x:1 if x in ["Medium","High"] else 0)

    # 3 Data Exploration
    report = data_exploration(df)
    print(report)

    # 4 Model Build
    models, X_train, X_test, y_train, y_test = model_build(df)

    # 5 Model Evaluation
    model_evalution(models, X_train, X_test, y_train, y_test)



if __name__ == "__main__":
    main()

(                       feature        mean  median  quartile_1  quartile_3  \
0                   Student_ID  353.000000   353.0       177.0       529.0   
1                          Age   20.659574    21.0        19.0        22.0   
2        Avg_Daily_Usage_Hours    4.918723     4.8         4.1         5.8   
3        Sleep_Hours_Per_Night    6.868936     6.9         6.0         7.7   
4          Mental_Health_Score    6.226950     6.0         5.0         7.0   
5  Conflicts_Over_Social_Media    2.849645     3.0         2.0         4.0   
6               Addicted_Score    6.436879     7.0         5.0         8.0   
7                     Addicted    0.975887     1.0         1.0         1.0   

     IQR  lower_whisker  upper_whisker  outlier_count     std_dev  \
0  352.0        -351.00        1057.00              0  203.660256   
1    3.0          14.50          26.50              0    1.399217   
2    1.7           1.55           8.35              3    1.257395   
3    1.7           3

c:\SocialMediaAddiction_Analysis\.venv\Lib\site-packages\xgboost\training.py:200: UserWarning: [21:29:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
